# 1. Setup

Run this cell first to install SLEAP.


## Install

In [ ]:
!pip install sleap==1.2.0a2

## Test

In [ ]:
#@title SLEAP and system versions: { display-mode: "form" }
import sleap
sleap.versions()
sleap.system_summary()

# 2. Upload data
Select the `.zip` file generated by going to: **SLEAP** → **Predict** → **Run Training...** → **Export training job package...**

In [ ]:
#@title Select training job package { display-mode: "form" }

from google.colab import files
import shutil
import yaml

uploaded = files.upload()
zip_file = list(uploaded.keys())[0]
print("Uploaded:", zip_file)

shutil.unpack_archive(zip_file)
with open("jobs.yaml", "r") as f:
    jobs = yaml.load(f, yaml.Loader)

print(f"Found {len(jobs['training'])} training job(s) and {len(jobs['inference'])} inference job(s).")

# 3. Run training

## TensorBoard monitoring

In [ ]:
#@title Launch TensorBoard { run: "auto", display-mode: "form" }
# enable_tensorboard = True #@param {type:"boolean"}
# if enable_tensorboard:
%load_ext tensorboard
%tensorboard --logdir "models"

## Train

In [ ]:
#@title Training { display-mode: "form" }
for job in jobs["training"]:
    cfg_path = job["cfg"]
    labels_path = job["train_labels"]
    !sleap-train {cfg_path} {labels_path} --tensorboard

# 4. Run inference

In [ ]:
for job in jobs["inference"]:
    models = " ".join([f"-m {m}" for m in job["models"]])
    suggested = "--only-suggested-frames" if job["only_suggested_frames"] else ""
    output_path = job["output_path"]
    data_path = job["data_path"]
    !sleap-track {models} {suggested} -o {output_path} {data_path}

# 5. Download results

In [ ]:
#@title Download inference results
#@markdown Run this cell to download the inference results. Predictions can be imported into a labeling project from **File** → **Merge into Project...**

preds = [job["output_path"] for job in jobs["inference"]]
if len(preds) == 1:
    files.download(preds[0])
else:
    # TODO: copy to tmpdir and zip
    pass

In [ ]:
#@title Download trained models
#@markdown Run this cell to download the trained models.

# models = [job["run_path"] for job in jobs["training"]]
shutil.make_archive(base_name="models", format="zip", base_dir="models")
files.download("models.zip")